<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifier/ADAPMIT/CPU_ADAPMIT_SentenceT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install optuna
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

[codecarbon INFO @ 15:43:11] Energy consumed for RAM : 0.000137 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:43:11] Energy consumed for all GPUs : 0.000123 kWh. Total GPU Power : 9.28099058829887 W
[codecarbon INFO @ 15:43:11] Energy consumed for all CPUs : 0.000565 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:43:11] 0.000825 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:43:56] Energy consumed for RAM : 0.000265 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:43:56] Energy consumed for all GPUs : 0.000414 kWh. Total GPU Power : 25.441500917784857 W
[codecarbon INFO @ 15:43:56] Energy consumed for all CPUs : 0.001096 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:43:56] 0.001776 kWh of electricity used since the beginning.


In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 18:21:42] [setup] RAM Tracking...
[codecarbon INFO @ 18:21:42] [setup] GPU Tracking...
[codecarbon INFO @ 18:21:42] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:21:42] [setup] CPU Tracking...
[codecarbon WARNING @ 18:21:42] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:21:43] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 18:21:43] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 18:21:43] >>> Tracker's metadata:
[codecarbon INFO @ 18:21:43]   Platform system: Linux-5.15.0-1040-azure-x86_64-with-glibc2.10
[codecarbon INFO @ 18:21:43]   Python version: 3.8.5
[codecarbon INFO @ 18:21:43]   CodeCarbon version: 2.3.2
[codecarbon INFO @ 18:21:43]   Available RAM : 27.406 GB
[codecarbon INFO @ 18:21:43]   CPU count: 4
[codecarbon INFO @ 18:21:43]   CPU model: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 18:21:

In [ ]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

2024-01-20 18:21:09.028332: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 18:21:12.227567: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-20 18:21:12.227597: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-20 18:21:18.232984: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

# Manual Inputs


In [ ]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
filename="policy_classification.json"


### Define labels ##################
label_names= ['AdaptationLabel','MitigationLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "sentence-transformers/all-mpnet-base-v2"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "ADAPMIT-multilabel-mpnet"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=4.08e-5   #4.04e-5
batch_size = 8
num_train_epochs= 10
weight_decay=0.021
warmup_steps = 300
gradient_accumulation_steps = 2

# Automated

## Dataset

In [ ]:
# Load Dataset from Hugging Face
# Load Dataset from Hugging Face
try:
    dataset = load_dataset(repo_id, data_files=filename,
                             use_auth_token=True )
    data = pd.DataFrame(dataset['train'])
except:
    from huggingface_hub import hf_hub_download
    hf_hub_download(repo_id, filename=filename,
                    repo_type="dataset",use_auth_token=True,
    local_dir= 'data/')
    data = pd.read_json('data/policy_classification.json')

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = data[data.split== 'Train'].reset_index(drop=True)
test_df = data[data.split=='Test'].reset_index(drop=True)

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

Using custom data configuration GIZ--policy_classification-cdc4b717b1bd720b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Failed to read file '/home/azureuser/.cache/huggingface/datasets/downloads/dee24a3afc63cd87c4a0a9517165e43822030f22df905fa089a2628e8d786ca4' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


{0: 'AdaptationLabel', 1: 'MitigationLabel'}


In [ ]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 12538
0 . AdaptationLabel : 5439
1 . MitigationLabel : 6659

 Classes Representation in Test Dataset: 1469
0 . AdaptationLabel : 611
1 . MitigationLabel : 786


In [ ]:
for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))

# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}
for label in label_names:
    positive_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==True))
    negative_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==False))
print("Positive Weights",'\n',positive_weights)
print("Negative Weights",'\n',negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . AdaptationLabel : 5439
1 . MitigationLabel : 6659
Positive Weights 
 {'AdaptationLabel': 1.1526015811730097, 'MitigationLabel': 0.9414326475446764}
Negative Weights 
 {'AdaptationLabel': 0.8830821242428512, 'MitigationLabel': 1.0663378125531553}


[codecarbon INFO @ 18:22:13] Energy consumed for RAM : 0.000086 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:22:13] Energy consumed for all GPUs : 0.000133 kWh. Total GPU Power : 22.69462440102435 W
[codecarbon INFO @ 18:22:13] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:22:13] 0.000573 kWh of electricity used since the beginning.


`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [ ]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Parameter 'function'=<function tokenize_and_encode at 0x7f529d2359d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Training data: 12538
Validation data: 1469


  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

[codecarbon INFO @ 18:22:28] Energy consumed for RAM : 0.000128 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:22:28] Energy consumed for all GPUs : 0.000239 kWh. Total GPU Power : 25.411914055331113 W
[codecarbon INFO @ 18:22:28] Energy consumed for all CPUs : 0.000532 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:22:28] 0.000899 kWh of electricity used since the beginning.


  0%|          | 0/12538 [00:00<?, ?ex/s]

  0%|          | 0/1469 [00:00<?, ?ex/s]

## Metrics

In [ ]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [ ]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "linear",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

## Custom Trainer

In [ ]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss


# Training

In [ ]:
multi_trainer =  Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)


[codecarbon INFO @ 18:22:47] [setup] RAM Tracking...
[codecarbon INFO @ 18:22:47] [setup] GPU Tracking...
[codecarbon INFO @ 18:22:47] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:22:47] [setup] CPU Tracking...
[codecarbon WARNING @ 18:22:47] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon WARNING @ 18:22:48] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 18:22:48] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 18:22:48] >>> Tracker's metadata:
[codecarbon INFO @ 18:22:48]   Platform system: Linux-5.15.0-1040-azure-x86_64-w

In [ ]:
multi_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You're using a MPNetTokenize

Epoch,Training Loss,Validation Loss,Precision-micro,Precision-samples,Precision-weighted,Recall-micro,Recall-samples,Recall-weighted,F1-micro,F1-samples,F1-weighted
1,0.377400,0.277775,0.890063,0.848877,0.889966,0.904080,0.845133,0.904080,0.897017,0.843204,0.896827
2,0.234900,0.260395,0.908104,0.847856,0.913150,0.898354,0.839687,0.898354,0.903203,0.840481,0.902137
3,0.171400,0.231205,0.917555,0.865895,0.919103,0.924123,0.862832,0.924123,0.920827,0.860676,0.920571
4,0.125000,0.234381,0.909534,0.870320,0.909552,0.935576,0.873383,0.935576,0.922371,0.867257,0.922309
5,0.087700,0.257381,0.921986,0.870320,0.922263,0.930565,0.867597,0.930565,0.926256,0.865668,0.926192
6,0.059200,0.305635,0.918785,0.867597,0.918845,0.931281,0.869299,0.931281,0.924991,0.864080,0.925011
7,0.041900,0.316437,0.918613,0.858407,0.918638,0.929134,0.865555,0.929134,0.923843,0.857499,0.923833
8,0.027000,0.357697,0.911111,0.864874,0.911286,0.939155,0.874404,0.939155,0.924921,0.864988,0.924955
9,0.015800,0.372125,0.903491,0.861130,0.903499,0.944882,0.877468,0.944882,0.923723,0.864080,0.923727
10,0.009400,0.383936,0.913889,0.867937,0.913900,0.942019,0.876106,0.942019,0.927741,0.867937,0.927744


[codecarbon INFO @ 18:22:58] Energy consumed for RAM : 0.000214 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:22:58] Energy consumed for all GPUs : 0.000465 kWh. Total GPU Power : 28.75058763256372 W
[codecarbon INFO @ 18:22:58] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:22:58] 0.001565 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:23:09] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:23:09] Energy consumed for all GPUs : 0.000254 kWh. Total GPU Power : 61.044040820347426 W
[codecarbon INFO @ 18:23:09] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:23:09] 0.000474 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:23:13] Energy consumed for RAM : 0.000257 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:23:13] Energy consumed for all GPUs : 0.000756 kWh. Total GPU Power : 69.74467086426094 W

TrainOutput(global_step=7840, training_loss=0.11496771932864676, metrics={'train_runtime': 10206.8765, 'train_samples_per_second': 12.284, 'train_steps_per_second': 0.768, 'total_flos': 2.47416480907776e+16, 'train_loss': 0.11496771932864676, 'epoch': 10.0})

In [ ]:
tracker.stop()

[codecarbon INFO @ 21:13:00] Energy consumed for RAM : 0.029327 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:13:00] Energy consumed for all GPUs : 0.186666 kWh. Total GPU Power : 38.48961348302786 W
[codecarbon INFO @ 21:13:00] Energy consumed for all CPUs : 0.121320 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:13:00] 0.337313 kWh of electricity used since the beginning.


0.12327996750145848

In [ ]:
multi_trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/4.14k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1705774974.prashant-t4-core4.4200.0:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

'https://huggingface.co/ppsingh/ADAPMIT-multilabel-mpnet/tree/main/'

In [ ]:
log_results = multi_trainer.state.log_history
print(log_results)

import json
with open("classifier_info/ADAPMIT_mpnet_logs.json", "w") as fp:
    json.dump(log_results , fp)

[{'loss': 0.3774, 'learning_rate': 3.818100795755968e-05, 'epoch': 1.0, 'step': 784}, {'eval_loss': 0.2777751386165619, 'eval_precision-micro': 0.8900634249471459, 'eval_precision-samples': 0.8488767869298843, 'eval_precision-weighted': 0.8899664327317055, 'eval_recall-micro': 0.9040801717967072, 'eval_recall-samples': 0.8451327433628318, 'eval_recall-weighted': 0.9040801717967072, 'eval_f1-micro': 0.8970170454545455, 'eval_f1-samples': 0.8432039936464716, 'eval_f1-weighted': 0.8968269579391615, 'eval_runtime': 33.6236, 'eval_samples_per_second': 43.69, 'eval_steps_per_second': 5.472, 'epoch': 1.0, 'step': 784}, {'loss': 0.2349, 'learning_rate': 3.393867374005305e-05, 'epoch': 2.0, 'step': 1568}, {'eval_loss': 0.2603946030139923, 'eval_precision-micro': 0.9081041968162084, 'eval_precision-samples': 0.84785568413887, 'eval_precision-weighted': 0.9131501128536955, 'eval_recall-micro': 0.898353614889048, 'eval_recall-samples': 0.8396868618107556, 'eval_recall-weighted': 0.898353614889048,

# Predict and Evaluate

## Trainer

In [ ]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.50
y_pred = (y_prob>=thresh).bool()

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

AdaptationLabel
[[801  57]
 [ 36 575]] 

MitigationLabel
[[616  67]
 [ 45 741]] 



In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,AdaptationLabel,0.909810,0.941080,0.925181,611.0
1,MitigationLabel,0.917079,0.942748,0.929737,786.0
2,micro avg,0.913889,0.942019,0.927741,1397.0
3,macro avg,0.913445,0.941914,0.927459,1397.0
4,weighted avg,0.913900,0.942019,0.927744,1397.0
5,samples avg,0.867937,0.876106,0.867937,1397.0
